In [5]:
import pandas as pd
# sklearn
import minsearch

In [11]:
!pip install python-dotenv

In [12]:
import os
import dotenv
dotenv.load_dotenv()

True

#  Ingestion

In [2]:
import json

with open('../Data/documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [6]:
import minsearch

index = minsearch.Index(
    text_fields=['intent', 'question', 'response', 'category'],
    keyword_fields=['id']
)

In [7]:
index.fit(documents)

# RAG flow

In [13]:
from openai import OpenAI

client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ["HF_TOKEN"],
)

In [14]:
def search(query):
    boost = {'instruction': 3.0, 'intent': 0.5}

    results = index.search(
        query=query,
        filter_dict={'category': 'CLAIMS'},
        boost_dict=boost,
        num_results=5
    )

    return results
def build_prompt(query, search_results):
    prompt_template = """
    You are a customer support assistant. Answer the following question based on the information provided in the CONTEXT from our knowledge base. 

    Use only the facts from the CONTEXT to respond accurately.

    QUESTION: {instruction}

    CONTEXT:
    {context}
    """.strip()


    context = ""
    
    for doc in search_results:
        context = context + f"intent: {doc['intent']}\\question: {doc['question']}\\nanswer: {doc['response']}\\n\\n"
    
    prompt = prompt_template.format(instruction=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model="openai/gpt-oss-120b:fireworks-ai", # 'gpt-4o-mini'
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
      
    return response

In [15]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer.choices[0].message.content

In [16]:
question = 'I need assistance to shop some articles.'
answer = rag(question)
print(answer)

I’m sorry, but the information I have available only covers assistance with insurance claims and settlement offers. I don’t have any details on shopping for articles. If you need help with filing a claim or handling a settlement, I’d be happy to provide that guidance. Otherwise, you may need to contact the appropriate department or service that handles shopping assistance.
